# Milestone project 1 https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

Use tensorflow mixed percision training using both 32 bit and 64 bit to run faster.


In [1]:
## Check GPU, Need to use T4
#We need a GPU score 7.0+ for mixed precision training.
!nvidia-smi

Fri Jan 12 12:06:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:

import tensorflow as tf
print(tf.__version__)

# Get helper_functions.py script from course GitHub
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions we're going to use
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir, compare_historys

2.15.0
--2024-01-12 12:06:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-12 12:06:16 (88.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Use Tensorflow datasets to download data
we can use Tensorflow datasets to download data from https://www.tensorflow.org/datasets/catalog/food101#:~:text=This%20dataset%20consists%20of%20101,contain%20some%20amount%20of%20noise.


In [3]:
import tensorflow_datasets as tfds


In [4]:
# List all available datasets
datasets_list = tfds.list_builders() # gets all the available datsets in tfds
print("food101" in datasets_list) # is our target dataset in the list of TFDS dataset

True


In [ ]:
# Load in the data ( takes about 5-6 mins)
# Some datasets may be 100 Gb
#output sometime doesnt get updated completely


(train_data, test_data), ds_info = tfds.load(name ='food101',
                                             split=['train', 'validation'],
                                             shuffle_files =  False,
                                             as_supervised=True, # data gets returned in tuple fomat (data, label)
                                             with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
len(test_data)
test_data2 = test_data

In [ ]:
# Features of food101 from TFDS
ds_info.features

In [ ]:
# get the class names
class_names = ds_info.features['label'].names
class_names[:10]

## Exploring the Food101 data from TensorFlow Datasets

To become one with our data, we want to find:
* Class names
* Image shape (usually image input shape)
* The datatype of our input
* Number of images
* Number of classes / labels ( are the labels one-hot encoded or label encoded?)


In [ ]:
# Take one sample of the train data
train_one_sample =  train_data.take(1) # samples are in format (image_tensor, label)

# How does one sample look?
train_one_sample

In [ ]:
# Output info about training sample
for image, label in train_one_sample:
  print("Image shape: ", image.shape)
  print("Label: ", label)
  print("Datatype: ", image.dtype)
  print(class_names[label])

In [ ]:
import matplotlib.pyplot as plt
# since the label isnt one hot encoded we may need to use the sparse categorical crossentropy
plt.figure(figsize =(16, 10))
plt.imshow(image)

In [ ]:
# what are the min and max values of our image?
tf.reduce_min(image), tf.reduce_max(image)

# NN likes data type numbers between 0 and 1, so we should convert it by /255.

## Create preprocessing functions for our data
NN perform best when data is in a certain way ( batched and normalized)
however, not all data comes like this
so in order o get it NN ready, we write preprocessing functions and map it to our data.

What do we know already about ou data?
* In 'uint8' datatype
comprise of different size tensors
* Not scaled.

Models like:
* Dtype as float32 or float16 for mixed precision
* For batches, tensorflow likes all the tensors within a batch to be in the same size.
* Scaled values between 0 & 1 (normalized) performs better.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

def preprocess_img(image, label, img_shape =224):
  """
  converts image datatype from uint8 to float32 and reshape
  image to [img_shape, img_shape, color=3]
  """
  image = tf.cast(image, tf.float32)
  #image = image/255.0
  image = tf.image.resize(image, (img_shape, img_shape))
  return image, label

In [ ]:
preprocessed_img = preprocess_img(image, label)[0]
print(f'Processed img: {preprocessed_img.shape}, Dtype: {preprocessed_img.dtype}')

# Batch and prepare datasets

make data input pipeline really fast.
largest problem usually loading data.

In [ ]:
# Map preprocessing function to data
# Parallelize it

train_data2 = train_data
train_data = train_data.map(map_func =preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)


# Create model callbacks
explore checkpoints and training log tracking


In [ ]:
import os

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                   save_best_only = True,
                                   monitor = 'val_acc',
                                   save_weights_only=True,
                                   verbose=0)

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
mixed_precision.global_policy()

In [ ]:
from tensorflow.keras import layers

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = layers.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x) # want one output neuron per class
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)



In [ ]:
# Turn off all warnings except for errors
tf.get_logger().setLevel('ERROR')

# Fit the model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     epochs=3,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=checkpoint_callback)

In [ ]:
model_results =  model.evaluate(test_data)

In [ ]:
os.listdir(checkpoint_dir)

In [ ]:
#load the weights
# model.load_weights(checkpoint_path)

In [ ]:
model.save("drive/MyDrive/bigfood")

In [ ]:
load_model =  tf.keras.models.load_model("drive/MyDrive/bigfood")

In [ ]:
for layer in load_model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
loaded_results = load_model.evaluate(test_data)

In [ ]:
import numpy as np
np.isclose(loaded_results, model_results)

In [ ]:
load_model.summary()

In [ ]:
load_model.trainable = True

In [ ]:
for layer in load_model.layers:
  print(layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                   save_best_only = True,
                                   monitor = 'val_loss',
                                   save_weights_only=True,
                                   verbose=0)

In [ ]:
load_model.compile(loss ="sparse_categorical_crossentropy",
                   optimizer = tf.keras.optimizers.Adam(0.0001),
                   metrics = "accuracy")

In [ ]:
history_1 =  load_model.fit(train_data,
                            epochs = 100,
                            steps_per_epoch = len(train_data),
                            validation_data = test_data,
                            validation_steps = int(0.15 * len(test_data)),
                            callbacks = [checkpoint_callback, earlystopping])

In [ ]:
load_model.evaluate(test_data)

In [ ]:

# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

In [ ]:
dbourke_model =  tf.keras.models.load_model("/content/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
before_results = dbourke_model.evaluate(test_data)

In [ ]:
dbourke_model.summary()

In [ ]:
dbourke_model.trainable = True

In [ ]:
for layer in dbourke_model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [ ]:
dbourke_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(0.0001),
                      metrics = 'accuracy')



In [ ]:
history_2 = dbourke_model.fit(train_data,
                  epochs=100,
                  steps_per_epoch=len(train_data),
                  validation_data=test_data,
                  validation_steps=int(0.15 * len(test_data)),
                  callbacks = [checkpoint_callback, earlystopping,reduce_lr])

In [ ]:
top_results = dbourke_model.evaluate(test_data
                       )

In [ ]:
predictions = dbourke_model.predict(test_data)

In [ ]:
pred_classes = predictions.argmax(axis=1)

In [ ]:
pred_classes[:10]

In [ ]:
len(pred_classes)

In [ ]:
y_labels = []
for images, labels in test_data.unbatch():
  y_labels.append(labels.numpy())


In [ ]:
y_labels[:10]

In [ ]:
len(y_labels)

In [ ]:
pred_classes

In [ ]:
from helper_functions import make_confusion_matrix

pred_list = tf.data.Dataset.from_tensor_slices(list(predictions))
# pred_list = pred_list.batch(1)

In [ ]:
class_names = ds_info.features['label'].names

In [ ]:
# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_true = y_labels,
                      y_pred = pred_classes,
                      classes = class_names,
                      figsize = (100,100),
                      text_size=20,
                      norm=False,
                      savefig=True)